In [1]:
%pip install torch==1.11.0+cu113 torchdata==0.3.0 torchtext==0.12.0 -f https://download.pytorch.org/whl/cu113/torch_stable.html
%pip install spacy tqdm
%pip install datasets
!python -m spacy download en_core_web_sm

In [20]:
from datasets import load_dataset, load_metric
dataset = load_dataset("surrey-nlp/PLOD-CW")

In [21]:
train = dataset["train"]
valid = dataset["validation"]
test = dataset["test"]


train_tokens = train["tokens"]
train_labels = train["ner_tags"]
valid_tokens = valid["tokens"]
valid_labels = valid["ner_tags"]
test_tokens = test["tokens"]
test_labels = test["ner_tags"]	



In [22]:
#find maximum sequence length
max_length = 0
for i in train_tokens:
  print(len(i))
  break

15


In [23]:
print(train_tokens[0])
print(train_tokens[1])
print(train_labels[0])
print(train_labels[1])


['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.']
['The', 'following', 'physiological', 'traits', 'were', 'measured', ':', 'stomatal', 'conductance', '(', 'gs', ',', 'mol', 'H2O', 'm-2', 's-1', ')', ',', 'transpiration', 'rate', '(', 'E', ',', 'mmol', 'H2O', 'm-2', 's-1', ')', ',', 'net', 'photosynthetic', 'rate', '(', 'PN', ',', 'μmol', 'm-2', 's-1', ')', 'and', 'intercellular', 'CO2', 'concentration', 'CO2', '(', 'Ci', ',', 'μmol', 'm-2', 's-1', ')', '.']
['B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O']
['B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'B-AC', 'B-O', 'B-AC', 'B

In [24]:
#convert the token lists to sentences
def tokens_to_sentences(tokens):
	sentences = []
	for token in tokens:
		sentences.append(" ".join(token))
	return sentences



In [25]:
train_sentences = tokens_to_sentences(train_tokens)
valid_sentences = tokens_to_sentences(valid_tokens)
test_sentences = tokens_to_sentences(test_tokens)

In [26]:
#find longest sentence
max_length = 0
longest = None
for i,v in enumerate(train_tokens):
	if len(v) > max_length:
		max_length = len(v)	
		longest =v
print(max_length)
print(" ".join(longest))


323
( EGF , epidermal growth factor ; TGF , transforming growth factor ; BTC , betacellulin ; HB - EGF , heparin - binding epidermal growth factor ( EGF)-like growth factor ; EREG , epiregulin ; NRG1 , neuregulin-1 ; NRG2 , neuregulin-2 ; NRG3 , neuregulin-3 ; NRG4 , neuregulin-4 ; PLCγ , phospholipase C type gamma ; CAMK2B , calcium / calmodulin dependent protein kinase ; PRKCB , Protein kinase C - beta ; STAT5 , Signal transducer and activator of transcription 5 ; src , Rous sarcoma virus gene ; CRK , C T10 regulator of a tyrosine kinase ; NCL , NCK Adaptor Protein 2 ; PTK2 , PTK2 protein tyrosine kinase 2 ; ABL2 , V - Abl Abelson Murine Leukemia Viral Oncogene Homolog 2 ; PAK2 , P21 ( RAC1 ) Activated Kinase 2 ; MAP2K4 , Mitogen - Activated Protein Kinase Kinase 4 ; MAPK10 , Mitogen - Activated Protein Kinase 10 ; SOS1 , SOS Ras / Rac Guanine Nucleotide Exchange Factor 1 ; Grb2 , Growth Factor Receptor Bound Protein 2 ; SHC4 , Src Homology 2 Domain - Containing - Transforming Protei

In [143]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

sentences = train_tokens
labels = train_labels
print(sentences[0])
print(labels[0])
token_tokenizer = Tokenizer(lower=True, oov_token="<UNK>")
token_tokenizer.fit_on_texts(sentences)
word_index = token_tokenizer.word_index
word_index["PAD"] = 0  # Ensure padding token is 0
sequences = token_tokenizer.texts_to_sequences(sentences)
max_seq_length = max(len(s) for s in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post', value=word_index["PAD"])

label_tokenizer = Tokenizer(lower=False)
label_tokenizer.fit_on_texts(labels)
label_index = label_tokenizer.word_index

print(len(label_index))
num_labels = len(label_index) + 1 
label_sequences = label_tokenizer.texts_to_sequences(labels)
padded_label_sequences = pad_sequences(label_sequences, maxlen=max_seq_length, padding='post', value=0)

label_mask = (padded_label_sequences != 0).astype(np.float32)

num_classes = len(label_index) + 1  # Assuming 0 is not used by label_index
one_hot_labels = np.zeros((padded_label_sequences.shape[0], padded_label_sequences.shape[1], num_classes))

for i, sequence in enumerate(padded_label_sequences):
    for j, label in enumerate(sequence):
        if label != 0:  # Only create one-hot vectors for non-padding labels
            one_hot_labels[i, j, label] = 1

one_hot_labels = np.array(one_hot_labels)




['For', 'this', 'purpose', 'the', 'Gothenburg', 'Young', 'Persons', 'Empowerment', 'Scale', '(', 'GYPES', ')', 'was', 'developed', '.']
['B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O']
4


In [141]:
example = label_sequences[0]
print(len(example))
example = one_hot_labels[0]

print(len(example))
print(example[15:])

15
323
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [146]:
# Define an RNN in TensorFlow/Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(word_index), output_dim=64, input_length=max_seq_length, mask_zero=True))
model.add(SimpleRNN(units=64, return_sequences=True))
model.add(TimeDistributed(Dense(num_labels, activation='softmax')))

# Compile the model
adam = Adam(learning_rate=2e-5)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, one_hot_labels, batch_size=32, epochs=10, sample_weight = label_mask)

Epoch 1/10


c:\Users\liamd\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


34/34 ━━━━━━━━━━━━━━━━━━━━ 14s 102ms/step - accuracy: 0.2006 - loss: 1.6036
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.1072 - loss: 1.5356
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.0786 - loss: 1.3981
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.0836 - loss: 1.2602
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.0892 - loss: 1.1395
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.0910 - loss: 1.0505
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.0937 - loss: 0.9734
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.0926 - loss: 0.9316
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.0923 - loss: 0.8889
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.0954 - loss: 0.8484


In [61]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import argmax

# Let's assume `new_sentences` is a list of new text data you want to predict.
new_sentences = train_sentences[:2]
new_labels = train_labels[:2]

# Tokenize the new sentences using the same tokenizer instance
new_sequences = token_tokenizer.texts_to_sequences(new_sentences)

# Pad the new sequences
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_seq_length, padding='post', value=word_index["PAD"])

# Predict classes with the model
predictions = model.predict(new_padded_sequences)

# Taking argmax to get the most likely class index
predicted_class_indices = argmax(predictions, axis=-1)
print(predicted_class_indices[1])
print(new_labels[1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[1 2 1 4 2 1 1 2 1 2 1 1 1 1 1 1 1 2 1 0 4 1 1 2 1 1 4 1 1 1 2 4 1 1 1 1 2
 1 1 3 1 2 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-LF', 'I-LF',

In [150]:
# test the test set against the model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

test_sequences = token_tokenizer.texts_to_sequences(test_tokens)
test_label_sequences = label_tokenizer.texts_to_sequences(test_labels)

# Pad the sequences to have the same length as the training data.
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_seq_length, padding='post', value=word_index["PAD"])

padded_test_label_sequences = pad_sequences(test_label_sequences, maxlen=max_seq_length, padding='post', value=label_index["PAD"])
one_hot_test_labels = [[to_categorical(i, num_classes=num_labels) for i in label] for label in padded_test_label_sequences]
one_hot_test_labels = np.array(one_hot_test_labels)

# Predict on the test data.
loss, accuracy  = model.evaluate(padded_test_sequences, one_hot_test_labels)
loss, accuracy



KeyError: 'PAD'